In [ ]:

from deepface import DeepFace

import numpy as np
# import pandas as pd
import os

from sklearn.decomposition import PCA
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.decomposition import PCA
# from sklearn.datasets import make_classification
# import torch
# from torch.utils.data import DataLoader, TensorDataset
# from qiskit import QuantumCircuit, Aer, transpile
# #from qiskit.providers.aer import AerSimulator
# from qiskit_machine_learning.neural_networks import TwoLayerQNN
# from qiskit_machine_learning.algorithms import NeuralNetworkClassifier
# from qiskit.utils import QuantumInstance
# from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
# #from qiskit.algorithms.optimizers import COBYLA
# from sklearn.metrics import accuracy_score

# only deep face ile face boundary çıkarma from folder Dataset/train
import cv2
def extract_faces_from_folder(folder_path, target_size=(64, 64)):
    X = []
    y = []
    label_map = {} 
    current_label = 0

    for subdir in os.listdir(folder_path):
        subdir_path = os.path.join(folder_path, subdir)
        if os.path.isdir(subdir_path):
            if subdir not in label_map:
                label_map[subdir] = current_label
                current_label += 1
            label = label_map[subdir]

            for filename in os.listdir(subdir_path):
                file_path = os.path.join(subdir_path, filename)
                try:
                    # Use DeepFace to detect and extract the face
                    face_img = DeepFace.represent(file_path,  enforce_detection=True, model_name="Facenet512", detector_backend='mtcnn')
                    for face in face_img:
                        print(face.get("facial_area"))
                        if face_img is not None:
                            X.append(face.get("embedding"))  
                            y.append(label)
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")

    return np.array(X), np.array(y)

def  MinMaxScaler():
    pass

N_SAMPLES = 1800       
N_FEATURES_INITIAL = 512 
N_CLASSES = 8         
N_QUBITS = 4           
SHOTS = 1024           
EPOCHS = 30            
LEARNING_RATE = 0.01   
BATCH_SIZE = 16        
# face boundary çıkarma ve
# DeepFace kullanarak yüzleri çıkar
X_train, y_train = extract_faces_from_folder('Dataset/train')
X_test, y_test = extract_faces_from_folder('Dataset/test')



2025-09-02 16:28:23.321268: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-02 16:28:23.321534: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-02 16:28:23.357659: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-02 16:28:24.201903: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

In [ ]:
np.save('X_faces_train.npy', X_train)
np.save('y_labels_train.npy', y_train)
np.save('X_faces_test.npy', X_test)
np.save('y_labels_test.npy', y_test)

In [ ]:
X_train = np.load('X_faces_train.npy')
y_train = np.load('y_labels_train.npy')

X_test = np.load('X_faces_test.npy')
y_test = np.load('y_labels_test.npy')


In [ ]:


from qiskit import QuantumCircuit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
from torch.utils.data import DataLoader, TensorDataset

if X_train.shape[0] != y_train.shape[0]:
    raise ValueError("Number of samples in X and y do not match!")

if X_test.shape[0] != y_test.shape[0]:
    raise ValueError("Number of samples in X_test and y_test do not match!")
if X_train.shape[1] != N_FEATURES_INITIAL:
    raise ValueError(f"Expected {N_FEATURES_INITIAL} features, but got {X_train.shape[1]}")

# --- Özellik Ölçekleme ---
pca = PCA(n_components=6)
X_pca = pca.fit_transform(X)
print(f"PCA sonrası Veri Boyutu (X_pca): {X_pca.shape}")
print(X_pca.size)


scaler = MinMaxScaler(feature_range=(0, np.pi))
X_scaled = scaler.fit_transform(X_pca)
print(f"Ölçeklendirilmiş Veri Boyutu (X_scaled): {X_scaled.shape}")

scaler = MinMaxScaler(feature_range=(0, np.pi))
X_scaled = scaler.fit_transform(X_pca)



# PyTorch ile çalışmak için verileri Tensor formatına dönüştürüyoruz.
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# DataLoader, verileri batch'ler halinde modele beslememizi sağlar.
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=BATCH_SIZE)

print(f"Eğitim verisi boyutu: {X_train.shape}")
print(f"Test verisi boyutu: {X_test.shape}")

from qiskit.circuit import ParameterVector

# --- Adım 3: Kuantum Veri Kodlama (Feature Map) ---
feature_params = ParameterVector('x', length=N_QUBITS)

# ŞİMDİ, bu parametreleri kullanacağımız boş devreyi oluşturuyoruz.
feature_map = QuantumCircuit(N_QUBITS, name="FeatureMap")
for i in range(N_QUBITS):
    feature_map.ry(feature_params[i], i)

# feature_map.draw('mpl', style='iqx') # Bu satır hala çalışır.
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit_machine_learning.neural_networks import EstimatorQNN
# --- Adım 4: Parametreli Kuantum Devresi (Ansatz) ---
ansatz = RealAmplitudes(N_QUBITS, reps=2, entanglement='linear')
ansatz.draw('mpl', style='iqx')


# Kuantum Devresini Oluşturma
qc = QuantumCircuit(N_QUBITS)
qc.compose(feature_map, inplace=True)
qc.compose(ansatz, inplace=True)
print("\nTam Kuantum Devresi (Feature Map + Ansatz):")
qc.draw('mpl', style='iqx')


# --- QNN'i Qiskit'te Tanımlama ---
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp

observable_strings = ['I'*i + 'Z' + 'I'*(N_QUBITS-1-i) for i in range(N_QUBITS)]

observables = [SparsePauliOp(s) for s in observable_strings]

qnn = EstimatorQNN(
    circuit=qc,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    observables=observables,  
    estimator=StatevectorEstimator() 
)
print(f"\nQNN Giriş Parametreleri: {qnn.input_params}")
print(f"QNN Ağırlık Parametreleri: {qnn.weight_params}")



No gradient function provided, creating a gradient function. If your Estimator requires transpilation, please provide a pass manager.


Başlangıç Veri Boyutu (X): (297, 512)
Etiket Boyutu (y): (297,)
Extracted 297 faces with 512 features each.
Labels shape: (297,)
PCA sonrası Veri Boyutu (X_pca): (297, 6)
1782
Ölçeklendirilmiş Veri Boyutu (X_scaled): (297, 6)
Eğitim verisi boyutu: (207, 6)
Test verisi boyutu: (90, 6)

Tam Kuantum Devresi (Feature Map + Ansatz):

QNN Giriş Parametreleri: [ParameterVectorElement(x[0]), ParameterVectorElement(x[1]), ParameterVectorElement(x[2]), ParameterVectorElement(x[3])]
QNN Ağırlık Parametreleri: [ParameterVectorElement(θ[0]), ParameterVectorElement(θ[1]), ParameterVectorElement(θ[2]), ParameterVectorElement(θ[3]), ParameterVectorElement(θ[4]), ParameterVectorElement(θ[5]), ParameterVectorElement(θ[6]), ParameterVectorElement(θ[7]), ParameterVectorElement(θ[8]), ParameterVectorElement(θ[9]), ParameterVectorElement(θ[10]), ParameterVectorElement(θ[11])]
